In [ ]:
import torch
import cv2 as cv
from ultralytics import YOLO
import numpy as np
import os
import json
from pathlib import Path
import shutil
import random
from datetime import datetime
import yaml
import time

In [ ]:
# 1단계 : 설치 확인 및 YOLO 테스트
def check_installation():
    """설치된 패키지들 확인"""
    print("=" * 50)
    print("🔍 설치 확인 중...")
    print("=" * 50)
    
    # 1. PyTorch 확인
    print(f"✅ PyTorch: {torch.__version__}")
    print(f"💻 Device: {'GPU' if torch.cuda.is_available() else 'CPU'}")
    
    # 2. OpenCV 확인
    print(f"✅ OpenCV: {cv.__version__}")
    
    # 3. 카메라 확인
    cap = cv.VideoCapture(0)
    if cap.isOpened():
        print("✅ 카메라: 사용 가능")
        cap.release()
    else:
        print("⚠️ 카메라: 접근 불가 (나중에 다시 확인)")
    
    print("\n🎯 YOLO 모델 테스트 중...")
    
    # 4. YOLO 모델 로드 테스트
    try:
        model = YOLO('yolov8n.pt')  # 자동으로 다운로드됨
        print("✅ YOLO 모델 로드 성공!")
        
        # 5. 더미 이미지로 추론 테스트
        dummy_image = np.zeros((640, 640, 3), dtype=np.uint8)
        results = model(dummy_image)
        print("✅ 모델 추론 테스트 성공!")
        
        return True
        
    except Exception as e:
        print(f"❌ YOLO 모델 테스트 실패: {e}")
        return False

def first_yolo_demo():
    """첫 번째 YOLO 데모 - 웹캠으로 일반 객체 탐지"""
    print("\n🚀 첫 번째 YOLO 데모를 시작합니다!")
    print("(일반 객체 탐지 - 아직 쓰레기 전용 모델은 아닙니다)")
    print("'q' 키를 눌러 종료하세요.")
    
    # YOLO 모델 로드
    model = YOLO('yolov8n.pt')
    
    # 웹캠 시작
    cap = cv.VideoCapture(0)
    
    if not cap.isOpened():
        print("❌ 카메라를 열 수 없습니다.")
        print("다음 단계에서는 이미지 파일로 테스트하겠습니다.")
        return
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # YOLO 추론
        results = model(frame, conf=0.5)
        
        # 결과 그리기
        annotated_frame = results[0].plot()
        
        # 화면에 표시
        cv.imshow('YOLO Demo - Press Q to quit', annotated_frame)
        
        # 'q' 키로 종료
        if cv.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv.destroyAllWindows()
    print("✅ 데모 완료!")

if __name__ == "__main__":
    # 설치 확인
    if check_installation():
        print("\n🎉 모든 설치가 완료되었습니다!")
        
        # 첫 데모 실행 여부 묻기
        print("\n첫 번째 YOLO 데모를 실행하시겠습니까?")
        print("(웹캠으로 일반 객체 탐지 - 사람, 자동차, 컵 등)")
        
        # 자동으로 실행 (실제로는 사용자가 선택)
        run_demo = input("실행하려면 Enter, 건너뛰려면 'n' 입력: ").lower()
        
        if run_demo != 'n':
            first_yolo_demo()
        
        print("\n✨ 1단계 완료! 2단계(데이터 준비)로 넘어갈 준비가 되었습니다.")
    else:
        print("\n❌ 설치에 문제가 있습니다. 에러 메시지를 확인해주세요.")

In [ ]:
# 2단계 : 기존 폴더 구조의 이미지를 YOLO 형식으로 변환하기

class ExistingDataConverter:
    """기존 폴더 구조의 데이터를 YOLO 형식으로 변환"""
    
    def __init__(self, source_path="trashnet", target_path="dataset"):
        self.source_path = Path(source_path)
        self.target_path = Path(target_path)
        
        # 폴더명을 클래스로 매핑
        self.class_mapping = {
            'glass': 0,        # 유리
            'metal': 1,        # 금속 (알루미늄 캔 등)
            'paper': 2,        # 종이
            'plastic': 3,      # 플라스틱
            'trash': 4         # 일반쓰레기
        }
        
        self.setup_yolo_structure()
        
    def setup_yolo_structure(self):
        """YOLO 형식 폴더 구조 생성"""
        dirs = [
            self.target_path / "images" / "train",
            self.target_path / "images" / "val", 
            self.target_path / "labels" / "train",
            self.target_path / "labels" / "val"
        ]
        
        for dir_path in dirs:
            dir_path.mkdir(parents=True, exist_ok=True)
        
        print(f"✅ YOLO 형식 폴더 구조 생성: {self.target_path}")
    
    def analyze_existing_data(self):
        """기존 데이터 분석"""
        print("🔍 기존 데이터 분석 중...")
        print("=" * 50)
        
        total_images = 0
        class_counts = {}
        
        for folder_name in self.class_mapping.keys():
            folder_path = self.source_path / folder_name
            
            if folder_path.exists():
                # 이미지 파일 확장자
                image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
                images = []
                
                for ext in image_extensions:
                    images.extend(list(folder_path.glob(f"*{ext}")))
                    images.extend(list(folder_path.glob(f"*{ext.upper()}")))
                
                count = len(images)
                class_counts[folder_name] = count
                total_images += count
                
                print(f"📁 {folder_name:<10}: {count:>4}장")
            else:
                print(f"⚠️ {folder_name:<10}: 폴더 없음")
                class_counts[folder_name] = 0
        
        print("-" * 50)
        print(f"📊 총 이미지: {total_images}장")
        print("=" * 50)
        
        return class_counts, total_images
    
    def convert_to_yolo_format(self, train_ratio=0.8):
        """기존 데이터를 YOLO 형식으로 변환"""
        print(f"🔄 YOLO 형식으로 변환 시작 (train/val 비율: {train_ratio:.1f}/{1-train_ratio:.1f})")
        
        total_converted = 0
        train_count = 0
        val_count = 0
        
        for folder_name, class_id in self.class_mapping.items():
            folder_path = self.source_path / folder_name
            
            if not folder_path.exists():
                print(f"⚠️ 폴더 없음: {folder_name}")
                continue
            
            # 이미지 파일 찾기
            image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
            images = []
            
            for ext in image_extensions:
                images.extend(list(folder_path.glob(f"*{ext}")))
                images.extend(list(folder_path.glob(f"*{ext.upper()}")))
            
            if not images:
                print(f"⚠️ 이미지 없음: {folder_name}")
                continue
            
            print(f"📁 {folder_name} 변환 중... ({len(images)}장)")
            
            # 이미지를 랜덤하게 섞어서 train/val 분할
            random.shuffle(images)
            split_idx = int(len(images) * train_ratio)
            
            train_images = images[:split_idx]
            val_images = images[split_idx:]
            
            # 훈련 데이터 처리
            for img_path in train_images:
                self.process_single_image(img_path, class_id, "train")
                train_count += 1
            
            # 검증 데이터 처리  
            for img_path in val_images:
                self.process_single_image(img_path, class_id, "val")
                val_count += 1
            
            total_converted += len(images)
            print(f"✅ {folder_name}: {len(train_images)}장(train) + {len(val_images)}장(val)")
        
        print("\n" + "=" * 50)
        print(f"🎉 변환 완료!")
        print(f"📊 총 변환: {total_converted}장")
        print(f"🏋️ 훈련 데이터: {train_count}장")
        print(f"🔍 검증 데이터: {val_count}장")
        print("=" * 50)
        
        return train_count, val_count
    
    def process_single_image(self, img_path, class_id, split):
        """개별 이미지를 처리하여 YOLO 형식으로 저장"""
        try:
            # 이미지 로드 및 확인
            image = cv.imread(str(img_path))
            if image is None:
                print(f"⚠️ 이미지 로드 실패: {img_path}")
                return
            
            # 새 파일명 생성
            new_filename = f"{class_id}_{img_path.stem}_{datetime.now().strftime('%H%M%S')}.jpg"
            
            # 이미지 복사
            target_img_path = self.target_path / "images" / split / new_filename
            cv.imwrite(str(target_img_path), image)
            
            # 전체 이미지를 하나의 바운딩 박스로 라벨링 (기본값)
            h, w = image.shape[:2]
            
            # YOLO 형식 라벨 생성 (전체 이미지를 객체로 가정)
            # center_x, center_y, width, height (모두 정규화)
            center_x = 0.5
            center_y = 0.5  
            width = 1.0
            height = 1.0
            
            # 라벨 파일 저장
            label_filename = f"{new_filename.rsplit('.', 1)[0]}.txt"
            label_path = self.target_path / "labels" / split / label_filename
            
            with open(label_path, 'w') as f:
                f.write(f"{class_id} {center_x} {center_y} {width} {height}\n")
            
        except Exception as e:
            print(f"❌ 처리 실패 {img_path}: {e}")

class InteractiveLabeler:
    """기존 변환된 데이터에 대해 정확한 바운딩 박스 라벨링"""
    
    def __init__(self, dataset_path="dataset"):
        self.dataset_path = Path(dataset_path)
        self.current_image = None
        self.current_image_path = None
        self.current_label_path = None
        self.bboxes = []
        self.drawing = False
        self.start_point = None
        
        # 라벨링할 이미지들 (train 폴더에서)
        self.image_files = list((self.dataset_path / "images" / "train").glob("*.jpg"))
        self.current_index = 0
        
        # 클래스 이름
        self.class_names = ['glass', 'metal', 'paper', 'plastic', 'trash']
        
        print(f"📝 {len(self.image_files)}개 이미지 라벨링 준비 완료")
    
    def mouse_callback(self, event, x, y, flags, param):
        """마우스 콜백 함수"""
        if event == cv.EVENT_LBUTTONDOWN:
            self.drawing = True
            self.start_point = (x, y)
        
        elif event == cv.EVENT_LBUTTONUP:
            if self.drawing and self.start_point:
                self.drawing = False
                end_point = (x, y)
                
                # 현재 파일의 클래스 ID 가져오기
                class_id = int(self.current_image_path.name.split('_')[0])
                
                # 바운딩 박스 추가
                x1, y1 = self.start_point
                x2, y2 = end_point
                
                # 좌표 정렬 (x1 < x2, y1 < y2)
                if x1 > x2:
                    x1, x2 = x2, x1
                if y1 > y2:
                    y1, y2 = y2, y1
                
                bbox = (class_id, x1, y1, x2, y2)
                self.bboxes.append(bbox)
                
                print(f"✅ 박스 추가: {self.class_names[class_id]} ({x1}, {y1}) to ({x2}, {y2})")
                self.draw_current_state()
    
    def draw_current_state(self):
        """현재 상태 그리기"""
        if self.current_image is None:
            return
            
        img_display = self.current_image.copy()
        
        # 바운딩 박스 그리기
        colors = [(0, 255, 255), (255, 0, 255), (255, 255, 0), (0, 255, 0), (255, 0, 0)]
        
        for bbox in self.bboxes:
            class_id, x1, y1, x2, y2 = bbox
            color = colors[class_id % len(colors)]
            
            cv.rectangle(img_display, (x1, y1), (x2, y2), color, 2)
            cv.putText(img_display, f"{self.class_names[class_id]}", (x1, y1-10),
                    cv.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        
        # 정보 표시
        info = f"[{self.current_index + 1}/{len(self.image_files)}] Boxes: {len(self.bboxes)}"
        cv.putText(img_display, info, (10, 30), cv.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        
        instructions = "Mouse: Draw | SPACE: Save | N: Next | P: Prev | C: Clear | Q: Quit"
        cv.putText(img_display, instructions, (10, img_display.shape[0] - 20), 
                cv.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        cv.imshow('Interactive Labeler', img_display)
    
    def load_current_image(self):
        """현재 이미지와 기존 라벨 로드"""
        if self.current_index >= len(self.image_files):
            return
        
        self.current_image_path = self.image_files[self.current_index]
        self.current_image = cv.imread(str(self.current_image_path))
        
        # 기존 라벨 파일 경로
        label_name = self.current_image_path.stem + ".txt"
        self.current_label_path = self.dataset_path / "labels" / "train" / label_name
        
        # 기존 라벨 로드
        self.load_existing_labels()
        self.draw_current_state()
        
        print(f"📁 로드: {self.current_image_path.name}")
    
    def load_existing_labels(self):
        """기존 라벨 파일 로드"""
        self.bboxes = []
        
        if not self.current_label_path.exists():
            return
        
        h, w = self.current_image.shape[:2]
        
        try:
            with open(self.current_label_path, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) == 5:
                        class_id = int(parts[0])
                        center_x = float(parts[1]) * w
                        center_y = float(parts[2]) * h
                        width = float(parts[3]) * w
                        height = float(parts[4]) * h
                        
                        x1 = int(center_x - width/2)
                        y1 = int(center_y - height/2)
                        x2 = int(center_x + width/2)
                        y2 = int(center_y + height/2)
                        
                        self.bboxes.append((class_id, x1, y1, x2, y2))
        except Exception as e:
            print(f"⚠️ 라벨 로드 실패: {e}")
    
    def save_labels(self):
        """라벨을 YOLO 형식으로 저장"""
        if not self.bboxes:
            print("⚠️ 저장할 라벨이 없습니다.")
            return
        
        h, w = self.current_image.shape[:2]
        
        with open(self.current_label_path, 'w') as f:
            for bbox in self.bboxes:
                class_id, x1, y1, x2, y2 = bbox
                
                center_x = (x1 + x2) / (2 * w)
                center_y = (y1 + y2) / (2 * h)
                width = abs(x2 - x1) / w
                height = abs(y2 - y1) / h
                
                f.write(f"{class_id} {center_x:.6f} {center_y:.6f} {width:.6f} {height:.6f}\n")
        
        print(f"💾 저장 완료: {len(self.bboxes)}개 박스")
    
    def start_labeling(self):
        """라벨링 시작"""
        if not self.image_files:
            print("❌ 라벨링할 이미지가 없습니다.")
            return
        
        cv.namedWindow('Interactive Labeler')
        cv.setMouseCallback('Interactive Labeler', self.mouse_callback)
        
        self.load_current_image()
        
        while True:
            key = cv.waitKey(1) & 0xFF
            
            if key == ord('q'):
                break
            elif key == ord(' '):  # 저장
                self.save_labels()
            elif key == ord('n'):  # 다음
                self.current_index = (self.current_index + 1) % len(self.image_files)
                self.load_current_image()
            elif key == ord('p'):  # 이전
                self.current_index = (self.current_index - 1) % len(self.image_files)
                self.load_current_image()
            elif key == ord('c'):  # 클리어
                self.bboxes = []
                self.draw_current_state()
        
        cv.destroyAllWindows()
        print("✅ 라벨링 완료!")

def create_dataset_yaml():
    """YAML 파일 생성"""
    yaml_content = """# Trash Classification Dataset
path: ./dataset
train: images/train
val: images/val

# Classes
nc: 5  # number of classes
names: ['glass', 'metal', 'paper', 'plastic', 'trash']
"""
    
    with open("dataset.yaml", "w", encoding="utf-8") as f:
        f.write(yaml_content)
    
    print("✅ dataset.yaml 파일이 생성되었습니다.")

def main():
    print("🔄 기존 데이터를 YOLO 형식으로 변환")
    print("=" * 50)
    
    while True:
        print("\n다음 중 선택하세요:")
        print("1️⃣ 데이터 분석 (현재 상태 확인)")
        print("2️⃣ YOLO 형식으로 변환")
        print("3️⃣ 정밀 라벨링 (바운딩 박스 수정)")
        print("4️⃣ dataset.yaml 생성")
        print("0️⃣ 다음 단계로")
        
        choice = input("선택 (0-4): ").strip()
        
        if choice == "1":
            converter = ExistingDataConverter()
            converter.analyze_existing_data()
        
        elif choice == "2":
            source_folder = input("원본 폴더 경로 (기본값: trashnet): ").strip()
            if not source_folder:
                source_folder = "trashnet"
            
            converter = ExistingDataConverter(source_folder)
            converter.convert_to_yolo_format()
        
        elif choice == "3":
            labeler = InteractiveLabeler()
            labeler.start_labeling()
        
        elif choice == "4":
            create_dataset_yaml()
        
        elif choice == "0":
            print("✅ 2단계 완료! 3단계(모델 훈련)로 넘어갑니다.")
            break
        
        else:
            print("잘못된 선택입니다.")

if __name__ == "__main__":
    main()

In [ ]:
# 3단계: YOLO 쓰레기 분류 모델 훈련

class WasteModelTrainer:
    """쓰레기 분류 YOLO 모델 훈련기"""
    
    def __init__(self, dataset_yaml="dataset.yaml"):
        self.dataset_yaml = dataset_yaml
        self.device = 'cpu'  # CPU 모드 고정
        
        # 훈련 결과 저장 경로
        self.results_path = Path("training_results")
        self.results_path.mkdir(exist_ok=True)
        
        print(f"🏋️ YOLO 모델 훈련 준비")
        print(f"💻 Device: {self.device}")
    
    def check_dataset(self):
        """데이터셋 상태 확인"""
        print("\n🔍 데이터셋 상태 확인 중...")
        print("=" * 50)
        
        # YAML 파일 확인
        if not Path(self.dataset_yaml).exists():
            print(f"❌ {self.dataset_yaml} 파일이 없습니다!")
            print("2단계에서 dataset.yaml을 생성해주세요.")
            return False
        
        # YAML 파일 읽기
        with open(self.dataset_yaml, 'r') as f:
            config = yaml.safe_load(f)
        
        dataset_path = Path(config['path'])
        train_images = dataset_path / config['train']
        val_images = dataset_path / config['val']
        train_labels = dataset_path / "labels" / "train"
        val_labels = dataset_path / "labels" / "val"
        
        # 폴더 존재 확인
        folders_ok = True
        for folder, name in [(train_images, "훈련 이미지"), (val_images, "검증 이미지"), 
                            (train_labels, "훈련 라벨"), (val_labels, "검증 라벨")]:
            if folder.exists():
                count = len(list(folder.glob("*")))
                print(f"✅ {name}: {count}개")
            else:
                print(f"❌ {name}: 폴더 없음")
                folders_ok = False
        
        # 클래스 정보
        print(f"📊 클래스 수: {config['nc']}")
        print(f"🏷️ 클래스: {config['names']}")
        
        print("=" * 50)
        
        if not folders_ok:
            print("❌ 필요한 폴더가 없습니다. 2단계를 먼저 완료해주세요.")
            return False
        
        print("✅ 데이터셋 준비 완료!")
        return True
    
    def train_model(self, model_size='n', epochs=50, batch_size=8, img_size=640):
        """모델 훈련 실행"""
        print(f"\n🚀 모델 훈련 시작!")
        print(f"📦 모델 크기: YOLOv8{model_size}")
        print(f"🔄 에포크: {epochs}")
        print(f"📦 배치 크기: {batch_size}")
        print(f"📏 이미지 크기: {img_size}")
        print("=" * 50)
        
        # 모델 로드
        model = YOLO(f'yolov8{model_size}.pt')
        print(f"✅ YOLOv8{model_size} 사전훈련 모델 로드 완료")
        
        # 훈련 시작 시간 기록
        start_time = time.time()
        
        try:
            # 훈련 실행
            results = model.train(
                data=self.dataset_yaml,     # 데이터셋 설정 파일
                epochs=epochs,              # 훈련 에포크
                imgsz=img_size,            # 이미지 크기
                batch=batch_size,          # 배치 크기
                device=self.device,        # CPU 사용
                project='runs/detect',     # 결과 저장 경로
                name='waste_classification', # 실험 이름
                save=True,                 # 모델 저장
                plots=True,                # 훈련 그래프 생성
                patience=10,               # 조기 종료 인내심
                save_period=10,            # 모델 저장 주기
                workers=2,                 # 데이터 로더 워커 (CPU에서는 낮게)
                verbose=True,              # 상세 로그
                val=True,                  # 검증 실행
                cache=False,               # 메모리 캐시 (CPU에서는 False)
                amp=False,                 # 자동 혼합 정밀도 (CPU에서는 False)
            )
            
            # 훈련 시간 계산
            training_time = time.time() - start_time
            hours = int(training_time // 3600)
            minutes = int((training_time % 3600) // 60)
            seconds = int(training_time % 60)
            
            print("\n🎉 훈련 완료!")
            print(f"⏰ 훈련 시간: {hours}시간 {minutes}분 {seconds}초")
            print(f"📁 결과 저장: runs/detect/waste_classification")
            
            # 최적 모델 복사
            best_model_path = Path("runs/detect/waste_classification/weights/best.pt")
            if best_model_path.exists():
                import shutil
                shutil.copy2(best_model_path, "best_waste_model.pt")
                print("✅ 최적 모델을 'best_waste_model.pt'로 복사했습니다.")
            
            return results
            
        except Exception as e:
            print(f"❌ 훈련 중 오류 발생: {e}")
            print("💡 해결 방법:")
            print("  - 배치 크기를 줄여보세요 (batch_size=4 또는 2)")
            print("  - 이미지 크기를 줄여보세요 (img_size=416)")
            print("  - 메모리가 부족할 수 있습니다.")
            return None
    
    def evaluate_model(self, model_path="best_waste_model.pt"):
        """훈련된 모델 평가"""
        print(f"\n📊 모델 성능 평가 중...")
        
        if not Path(model_path).exists():
            print(f"❌ 모델 파일을 찾을 수 없습니다: {model_path}")
            return
        
        try:
            # 모델 로드
            model = YOLO(model_path)
            
            # 검증 데이터셋으로 평가
            results = model.val(data=self.dataset_yaml)
            
            print("✅ 모델 평가 완료!")
            print(f"🎯 mAP50: {results.box.map50:.3f}")
            print(f"🎯 mAP50-95: {results.box.map:.3f}")
            
            # 클래스별 성능 출력
            if hasattr(results.box, 'ap_class_index') and results.box.ap_class_index is not None:
                print("\n📈 클래스별 성능:")
                with open(self.dataset_yaml, 'r') as f:
                    config = yaml.safe_load(f)
                
                class_names = config['names']
                for i, class_idx in enumerate(results.box.ap_class_index):
                    if i < len(results.box.ap50):
                        print(f"  {class_names[class_idx]}: {results.box.ap50[i]:.3f}")
            
            return results
            
        except Exception as e:
            print(f"❌ 평가 중 오류: {e}")
            return None
    
    def test_trained_model(self, model_path="best_waste_model.pt"):
        """훈련된 모델로 실시간 테스트"""
        print(f"\n🎥 실시간 쓰레기 탐지 테스트")
        
        if not Path(model_path).exists():
            print(f"❌ 모델 파일을 찾을 수 없습니다: {model_path}")
            return
        
        try:
            # 모델 로드
            model = YOLO(model_path)
            print("✅ 훈련된 모델 로드 완료")
            
            # 클래스 이름 로드
            with open(self.dataset_yaml, 'r') as f:
                config = yaml.safe_load(f)
            class_names = config['names']
            
            # 웹캠 시작
            cap = cv.VideoCapture(0)
            
            if not cap.isOpened():
                print("⚠️ 웹캠을 사용할 수 없습니다. 샘플 이미지로 테스트해보세요.")
                self.test_sample_images(model_path)
                return
            
            print("🎬 실시간 탐지 시작! 'q'를 눌러 종료하세요.")
            
            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                
                # 추론 실행
                results = model(frame, conf=0.3)  # 신뢰도 30% 이상
                
                # 결과 그리기
                annotated_frame = results[0].plot()
                
                # 탐지된 객체 정보 표시
                detections = results[0].boxes
                if detections is not None and len(detections) > 0:
                    info_text = f"Detected: {len(detections)} objects"
                    cv.putText(annotated_frame, info_text, (10, 30),
                            cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                
                cv.imshow('Waste Detection - Press Q to quit', annotated_frame)
                
                if cv.waitKey(1) & 0xFF == ord('q'):
                    break
            
            cap.release()
            cv.destroyAllWindows()
            print("✅ 실시간 테스트 완료!")
            
        except Exception as e:
            print(f"❌ 테스트 중 오류: {e}")
    
    def test_sample_images(self, model_path="best_waste_model.pt"):
        """샘플 이미지로 모델 테스트"""
        print("\n🖼️ 샘플 이미지 테스트")
        
        # 검증 이미지들로 테스트
        val_images_path = Path("dataset/images/val")
        if not val_images_path.exists():
            print("❌ 검증 이미지 폴더가 없습니다.")
            return
        
        sample_images = list(val_images_path.glob("*.jpg"))[:5]  # 최대 5장
        
        if not sample_images:
            print("❌ 테스트할 이미지가 없습니다.")
            return
        
        model = YOLO(model_path)
        
        for img_path in sample_images:
            print(f"🔍 테스트: {img_path.name}")
            
            # 이미지 로드
            image = cv.imread(str(img_path))
            
            # 추론
            results = model(image, conf=0.3)
            
            # 결과 표시
            annotated = results[0].plot()
            
            # 결과 저장
            output_path = self.results_path / f"test_{img_path.name}"
            cv.imwrite(str(output_path), annotated)
            
            print(f"  💾 결과 저장: {output_path}")
        
        print(f"✅ 샘플 테스트 완료! 결과: {self.results_path}")

def quick_training_setup():
    """빠른 훈련 설정"""
    print("⚡ 빠른 훈련 설정")
    print("=" * 30)
    print("CPU로 빠르게 테스트해보기 위한 설정입니다.")
    print("실제 사용할 때는 더 많은 에포크로 훈련하세요.")
    
    return {
        'model_size': 'n',      # nano (가장 가벼움)
        'epochs': 20,           # 적은 에포크 (빠른 테스트용)
        'batch_size': 4,        # 작은 배치 (메모리 절약)
        'img_size': 416         # 작은 이미지 크기 (빠른 처리)
    }

def production_training_setup():
    """실사용 훈련 설정"""
    print("🏭 실사용 훈련 설정")
    print("=" * 30)
    print("더 좋은 성능을 위한 설정입니다.")
    print("시간이 오래 걸리지만 더 정확한 모델이 만들어집니다.")
    
    return {
        'model_size': 's',      # small (균형잡힌 성능)
        'epochs': 100,          # 충분한 에포크
        'batch_size': 8,        # 적당한 배치 크기
        'img_size': 640         # 표준 이미지 크기
    }

def main():
    print("🏋️ 3단계: YOLO 쓰레기 분류 모델 훈련")
    print("=" * 50)
    
    trainer = WasteModelTrainer()
    
    # 데이터셋 확인
    if not trainer.check_dataset():
        print("\n❌ 데이터셋 문제가 있습니다. 2단계를 먼저 완료해주세요.")
        return
    
    while True:
        print("\n다음 중 선택하세요:")
        print("1️⃣ 빠른 훈련 (테스트용 - 20 에포크)")
        print("2️⃣ 실사용 훈련 (정식 - 100 에포크)")
        print("3️⃣ 커스텀 설정 훈련")
        print("4️⃣ 훈련된 모델 평가")
        print("5️⃣ 실시간 테스트")

        print("6️⃣ 샘플 이미지 테스트")
        print("0️⃣ 4단계로")
        
        choice = input("선택 (0-6): ").strip()
        
        if choice == "1":
            print("\n⚡ 빠른 훈련 시작!")
            config = quick_training_setup()
            trainer.train_model(**config)
        
        elif choice == "2":
            print("\n🏭 실사용 훈련 시작!")
            config = production_training_setup()
            trainer.train_model(**config)
        
        elif choice == "3":
            print("\n🛠️ 커스텀 설정")
            try:
                epochs = int(input("에포크 수 (기본 50): ") or "50")
                batch_size = int(input("배치 크기 (기본 4): ") or "4")
                img_size = int(input("이미지 크기 (기본 416): ") or "416")
                model_size = input("모델 크기 (n/s/m/l/x, 기본 n): ") or "n"
                
                trainer.train_model(
                    model_size=model_size,
                    epochs=epochs,
                    batch_size=batch_size,
                    img_size=img_size
                )
            except ValueError:
                print("❌ 잘못된 입력입니다.")
        
        elif choice == "4":
            trainer.evaluate_model()
        
        elif choice == "5":
            trainer.test_trained_model()
        
        elif choice == "6":
            trainer.test_sample_images()
        
        elif choice == "0":
            print("✅ 3단계 완료! 4단계(실시간 시스템)으로 넘어갑니다.")
            break
        
        else:
            print("❌ 잘못된 선택입니다.")

if __name__ == "__main__":
    main()

In [ ]:
# 4단계 : 쓰레기 탐지 시스템 

class CleanUIDetector:
    """쓰레기 탐지 시스템"""
    
    def __init__(self):
        # 모델 로드
        custom_model = Path("best_waste_model.pt")
        if custom_model.exists():
            self.model = YOLO(str(custom_model))
            self.model_type = "Custom Waste Model"
            print("✅ 커스텀 쓰레기 분류모델 로드")
        else:
            self.model = YOLO('yolov8n.pt')
            self.model_type = "General Object Detection"
            print("⚠️ 기본 클래스 사용")
        
        # 설정
        self.conf_threshold = 0.25
        self.frame_count = 0
        
        # 색상 설정 (BGR) - 더 예쁜 색상들
        self.box_colors = [
            (0, 0, 255),     # Red Box
            (0, 255, 0),     # Green Box  
            (255, 0, 0),     # Blue Box
            (0, 255, 255),   # Yellow Box
            (255, 0, 255),   # Magenta Box
            (255, 255, 0),   # Cyan Box
        ]
        
        self.color_names = [
            "Red Box", "Green Box", "Blue Box", 
            "Yellow Box", "Magenta Box", "Cyan Box"
        ]
    
    def detect_objects(self, frame):
        """객체 탐지"""
        results = self.model(frame, conf=self.conf_threshold, verbose=False)
        
        detections = []
        for result in results:
            boxes = result.boxes
            if boxes is not None:
                for box in boxes:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    conf = float(box.conf[0])
                    cls = int(box.cls[0])
                    
                    class_name = self.model.names[cls] if hasattr(self.model, 'names') else f'object_{cls}'
                    
                    detections.append({
                        'bbox': (x1, y1, x2, y2),
                        'confidence': conf,
                        'class_name': class_name,
                        'class_id': cls
                    })
        
        return detections
    
    def draw_clean_ui(self, frame, detections):
        """깔끔한 UI 그리기"""
        self.frame_count += 1
        
        # 상단 배경 (반투명)
        overlay = frame.copy()
        cv.rectangle(overlay, (0, 0), (frame.shape[1], 120), (0, 0, 0), -1)
        cv.addWeighted(overlay, 0.8, frame, 0.2, 0, frame)
        
        # 탐지 현황 계산
        class_counts = {}
        for det in detections:
            class_name = det['class_name']
            class_counts[class_name] = class_counts.get(class_name, 0) + 1
        
        # 왼쪽 기본 정보
        info_lines = [
            f"Objects Detected: {len(detections)}",
            f"Frame: {self.frame_count}",
            f"Confidence: {self.conf_threshold:.2f}",
            f"Model: {self.model_type}"
        ]
        
        for i, info in enumerate(info_lines):
            cv.putText(frame, info, (15, 25 + i * 22), 
                        cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        
        # 오른쪽 탐지 현황
        if detections:
            cv.putText(frame, "Detection Summary:", (frame.shape[1] - 280, 25),
                        cv.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
            
            y_offset = 50
            for class_name, count in class_counts.items():
                text = f"{class_name}: {count}"
                cv.putText(frame, text, (frame.shape[1] - 280, y_offset),
                            cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                y_offset += 22
        
        # 탐지된 객체들 그리기
        for i, det in enumerate(detections):
            x1, y1, x2, y2 = det['bbox']
            conf = det['confidence']
            class_name = det['class_name']
            
            # 색상 선택
            color = self.box_colors[i % len(self.box_colors)]
            
            # 바운딩 박스 (두꺼운 테두리)
            cv.rectangle(frame, (x1, y1), (x2, y2), color, 3)
            
            # 라벨 배경
            label = f'{class_name}: {conf:.2f}'
            (text_w, text_h), _ = cv.getTextSize(label, cv.FONT_HERSHEY_SIMPLEX, 0.7, 2)
            cv.rectangle(frame, (x1, y1 - text_h - 10), (x1 + text_w + 5, y1), color, -1)
            
            # 라벨 텍스트
            cv.putText(frame, label, (x1 + 2, y1 - 5), 
                        cv.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            
            # 중심점 (이쁜 원형)
            center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2
            cv.circle(frame, (center_x, center_y), 8, color, -1)
            cv.circle(frame, (center_x, center_y), 10, (255, 255, 255), 2)
            
            # 객체 번호
            cv.putText(frame, str(i + 1), (center_x - 5, center_y + 5),
                        cv.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        
        # 하단 상세 탐지 정보
        if detections:
            # 하단 배경
            bottom_height = min(100, 25 * len(detections) + 40)
            overlay2 = frame.copy()
            cv.rectangle(overlay2, (0, frame.shape[0] - bottom_height), 
                            (frame.shape[1], frame.shape[0] - 25), (0, 0, 0), -1)
            cv.addWeighted(overlay2, 0.8, frame, 0.2, 0, frame)
            
            # 상세 정보
            y_pos = frame.shape[0] - bottom_height + 20
            
            for i, det in enumerate(detections):
                conf = det['confidence']
                class_name = det['class_name']
                color_name = self.color_names[i % len(self.color_names)]
                
                # 색상 점
                color = self.box_colors[i % len(self.box_colors)]
                cv.circle(frame, (20, y_pos + 8), 8, color, -1)
                cv.circle(frame, (20, y_pos + 8), 10, (255, 255, 255), 1)
                
                # 텍스트
                text = f"[{color_name}] {class_name} {conf:.2f}"
                cv.putText(frame, text, (40, y_pos + 12), 
                            cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                y_pos += 25
        
        # 맨 하단 조작법
        cv.putText(frame, "SPACE: Save | +/-: Confidence | Q: Quit", 
                    (15, frame.shape[0] - 5), cv.FONT_HERSHEY_SIMPLEX, 0.6, (150, 150, 150), 1)
        
        return frame
    
    def run_detection(self):
        """실시간 탐지 실행"""
        cap = cv.VideoCapture(0)
        if not cap.isOpened():
            print("❌ 카메라를 열 수 없습니다.")
            return
        
        # 해상도 설정
        cap.set(cv.CAP_PROP_FRAME_WIDTH, 1280)
        cap.set(cv.CAP_PROP_FRAME_HEIGHT, 720)
        
        print("\n🎬 다중 객체 실시간 탐지 시작")
        print("🎮 조작법:")
        print("  SPACE: 현재화면 저장")
        print("  +/- key: 신뢰도 임계값 조정")
        print("  Q key: 종료")
        print("=" * 50)
        
        fps_counter = 0
        start_time = time.time()
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            # 객체 탐지
            detections = self.detect_objects(frame)
            
            # UI 그리기
            frame = self.draw_clean_ui(frame, detections)
            
            # FPS 표시
            fps_counter += 1
            if time.time() - start_time >= 1:
                fps = fps_counter / (time.time() - start_time)
                cv.putText(frame, f'FPS: {fps:.1f}', (frame.shape[1] - 100, 150), 
                            cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
                fps_counter = 0
                start_time = time.time()
            
            # 화면 표시
            cv.imshow('Waste Detection System', frame)
            
            # 키 입력 처리
            key = cv.waitKey(1) & 0xFF
            
            if key == ord('q'):
                break
            elif key == ord(' '):
                timestamp = time.strftime("%Y%m%d_%H%M%S")
                filename = f'detection_{timestamp}.jpg'
                cv.imwrite(filename, frame)
                print(f'📸 저장: {filename}')
            elif key == ord('+') or key == ord('='):
                old_conf = self.conf_threshold
                self.conf_threshold = min(0.9, self.conf_threshold + 0.05)
                print(f'🔍 임계값: {old_conf:.2f} → {self.conf_threshold:.2f}')
            elif key == ord('-'):
                old_conf = self.conf_threshold
                self.conf_threshold = max(0.05, self.conf_threshold - 0.05)
                print(f'🔍 임계값: {old_conf:.2f} → {self.conf_threshold:.2f}')
        
        cap.release()
        cv.destroyAllWindows()
        
        print(f"\n✅ 다중 객체 탐지완료")
        print(f"📊 총 처리 프레임: {self.frame_count}")

def main():
    print("🎯 다중 객체 탐지 시스템")
    print("=" * 50)
    
    detector = CleanUIDetector()
    detector.run_detection()

if __name__ == "__main__":
    main()